In [ ]:
Updating previous part with a user defined screeshot interval and saving it to the cloud.

In [2]:
import logging
import time
import threading
import os
import pyautogui
import boto3
from datetime import datetime
from pynput import mouse, keyboard
from pystray import Icon, MenuItem, Menu
from PIL import Image, ImageDraw
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import tkinter as tk
from tkinter import messagebox

# Configure logging for activity tracking (log to activity_log.txt file)
logging.basicConfig(filename="activity_log.txt", level=logging.INFO)

# Global variable to track activity status (whether the user is active or not)
is_active = False

# Function to create an icon for the system tray (the small icon shown in the taskbar)
def create_image():
    width = 64
    height = 64
    image = Image.new("RGB", (width, height), (255, 255, 255))  # Create a white image
    draw = ImageDraw.Draw(image)
    draw.rectangle([0, 0, width, height], fill=(0, 128, 255))  # Draw a blue square for the icon
    return image

# Function to log user activity (this will log mouse/keyboard activity to the log file)
def log_activity(action_type, action_detail):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} - {action_type}: {action_detail}"  # Format the log entry
    logging.info(log_entry)  # Write to the activity log file
    print(log_entry)  # Print the log to the console
    upload_log_to_s3()  # Upload the log to S3 after every log entry

# Functions for mouse activity tracking
def on_move(x, y):
    global is_active
    if not is_active:
        log_activity("Mouse Move", f"Mouse moved to {x}, {y}")
        is_active = True

def on_click(x, y, button, pressed):
    if pressed:
        log_activity("Mouse Click", f"Mouse clicked at {x}, {y}")

def on_scroll(x, y, dx, dy):
    log_activity("Mouse Scroll", f"Mouse scrolled at {x}, {y}, delta: ({dx}, {dy})")

# Function for keyboard activity tracking
def on_press(key):
    log_activity("Key Pressed", f"Key {key} pressed")

# Function to stop the listener when the system tray icon is quit
def on_quit(icon, item):
    icon.stop()

# Function to start listening for mouse and keyboard events
def start_listeners():
    # Start mouse listener (to track movement, clicks, and scrolls)
    mouse_listener = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
    mouse_listener.start()

    # Start keyboard listener (to track key presses)
    keyboard_listener = keyboard.Listener(on_press=on_press)
    keyboard_listener.start()

# System tray menu options (Right-click on the tray icon)
menu = Menu(MenuItem('Quit', on_quit))  # Option to quit the application
icon = Icon("Activity Tracker", create_image(), menu=menu)  # System tray icon with menu

# ActivityTracker class handles screenshot capturing and S3 file uploads
class ActivityTracker:
    def __init__(self, screenshot_interval=60, s3_bucket_name=None, s3_region='us-east-1'):
        self.screenshot_interval = screenshot_interval  # Interval in seconds for screenshot capturing
        self.s3_bucket_name = s3_bucket_name  # S3 bucket name to upload files
        self.s3_region = s3_region  # AWS region for S3
        self.s3_client = boto3.client('s3', region_name=self.s3_region)  # S3 client for file upload
        self.last_screenshot_time = time.time()  # Timestamp of the last screenshot taken

    # Function to capture a screenshot
    def capture_screenshot(self):
        current_time = time.time()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Timestamp for file naming
        screenshot_filename = f"screenshot_{timestamp}.png"
        screenshot = pyautogui.screenshot()  # Capture screenshot using pyautogui
        screenshot.save(screenshot_filename)  # Save the screenshot locally
        log_activity("Screenshot Captured", f"Screenshot captured and saved as {screenshot_filename}")  # Log activity
        self.upload_file_to_s3(screenshot_filename)  # Upload screenshot to S3
        os.remove(screenshot_filename)  # Delete the screenshot from the local system after uploading
        self.last_screenshot_time = current_time  # Update the last screenshot timestamp

    # Function to upload a file (e.g., screenshot) to S3
    def upload_file_to_s3(self, file_path):
        if not self.s3_bucket_name:
            print("No S3 bucket name provided.")
            return
        try:
            s3_object_key = f"screenshots/{os.path.basename(file_path)}"  # S3 key for the file
            self.s3_client.upload_file(file_path, self.s3_bucket_name, s3_object_key)  # Upload to S3
            print(f"Uploaded {file_path} to S3 bucket {self.s3_bucket_name} under the key {s3_object_key}")
        except (NoCredentialsError, PartialCredentialsError) as e:
            print(f"Error with AWS credentials: {e}")  # Handle AWS credential issues
        except Exception as e:
            print(f"Error uploading file to S3: {e}")  # Handle general upload errors
    
    # Function to upload the activity log to S3
    def upload_log_to_s3(self):
        log_file = "activity_log.txt"
        if os.path.exists(log_file):  # Check if the log file exists
            try:
                s3_object_key = f"logs/{log_file}"  # S3 key for the log file
                self.s3_client.upload_file(log_file, self.s3_bucket_name, s3_object_key)  # Upload log file to S3
                print(f"Uploaded {log_file} to S3 bucket {self.s3_bucket_name} under the key {s3_object_key}")
            except (NoCredentialsError, PartialCredentialsError) as e:
                print(f"Error with AWS credentials: {e}")  # Handle AWS credential issues
            except Exception as e:
                print(f"Error uploading log file to S3: {e}")  # Handle general upload errors

    # Function to periodically capture screenshots
    def capture_screenshots_periodically(self):
        while True:
            time.sleep(self.screenshot_interval)  # Wait for the specified interval
            self.capture_screenshot()  # Capture and upload a screenshot

# ConfigWindow class is a Tkinter-based configuration window for user settings
class ConfigWindow:
    def __init__(self, root, tracker):
        self.root = root
        self.root.title("Activity Tracker - Settings")  # Title of the settings window
        self.tracker = tracker  # The ActivityTracker instance

        # Screenshot interval input
        self.screenshot_interval_label = tk.Label(root, text="Screenshot Interval (seconds):")
        self.screenshot_interval_label.grid(row=0, column=0, padx=10, pady=10)
        
        self.screenshot_interval_entry = tk.Entry(root)
        self.screenshot_interval_entry.grid(row=0, column=1, padx=10, pady=10)
        self.screenshot_interval_entry.insert(0, str(self.tracker.screenshot_interval))  # Set default value

        # Checkbox to enable or disable screenshot capture
        self.enable_screenshot_var = tk.BooleanVar(value=True)  # Default to enabled
        self.enable_screenshot_check = tk.Checkbutton(root, text="Enable Screenshot Capture", variable=self.enable_screenshot_var)
        self.enable_screenshot_check.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

        # Button to save the settings
        self.save_button = tk.Button(root, text="Save Settings", command=self.save_settings)
        self.save_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    # Function to save settings when the "Save" button is clicked
    def save_settings(self):
        interval = self.screenshot_interval_entry.get()  # Get the screenshot interval value
        try:
            interval = int(interval)  # Try to convert to an integer
            if interval <= 0:
                raise ValueError("Interval must be a positive integer.")  # Check for positive values
            enable_screenshot = self.enable_screenshot_var.get()  # Get the screenshot enable/disable value
            self.tracker.screenshot_interval = interval  # Update the interval in the tracker
            messagebox.showinfo("Settings Saved", f"Settings saved:\nInterval: {interval} seconds\nScreenshot Capture: {'Enabled' if enable_screenshot else 'Disabled'}")
        except ValueError as e:
            messagebox.showerror("Invalid Input", str(e))  # Show error if invalid input is entered

# BackgroundAgent class to handle activity tracking and configuration integration
class BackgroundAgent:
    def __init__(self, s3_bucket_name, s3_region='us-east-1', screenshot_interval=60):
        self.s3_bucket_name = s3_bucket_name
        self.s3_region = s3_region
        self.screenshot_interval = screenshot_interval
        self.activity_tracker = ActivityTracker(screenshot_interval=screenshot_interval, s3_bucket_name=s3_bucket_name, s3_region=s3_region)
        self.config_window = None

    def start(self):
        # Start the activity listeners (mouse/keyboard tracking)
        start_listeners()

        # Start screenshot capturing periodically in the background
        screenshot_thread = threading.Thread(target=self.activity_tracker.capture_screenshots_periodically)
        screenshot_thread.daemon = True
        screenshot_thread.start()

        # Create and run the config window for settings
        self.config_window = tk.Tk()
        config_window = ConfigWindow(self.config_window, self.activity_tracker)
        self.config_window.mainloop()

# Start the background agent with appropriate settings
if __name__ == "__main__":
    agent = BackgroundAgent(s3_bucket_name="your-s3-bucket-name", s3_region="us-east-1", screenshot_interval=60)
    agent.start()  # Start the agent


2024-11-26 10:41:06 - Mouse Move: Mouse moved to 497, 468
2024-11-26 10:41:39 - Key Pressed: Key 'u' pressed


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\HAMID SAIFI\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\HAMID SAIFI\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_4824\2504092291.py", line 126, in capture_screenshots_periodically
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_4824\2504092291.py", line 90, in capture_screenshot
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_4824\2504092291.py", line 36, in log_activity
NameError: name 'upload_log_to_s3' is not defined


2024-11-26 10:42:05 - Screenshot Captured: Screenshot captured and saved as screenshot_20241126_104205.png
